In [ ]:
# Import Required Libraries
import json
import pandas as pd
import os
from DatabasePackage import assign_asset_to_player

def excel_to_assets_json(excel_file: str, json_file: str = None) -> dict:
    """
    Read an Excel file and produce a JSON file capturing the asset database
    grouped by Area -> Asset -> {land_price, house_price, rent: {...}}.

    Returns the generated assets dictionary and saves to JSON file.
    """

    # Read all sheets and take the first sheet
    excel_data = pd.read_excel(excel_file, sheet_name=None)
    df = list(excel_data.values())[0].copy()

    # Drop columns that are entirely NaN
    df = df.dropna(axis=1, how='all')

    # If the first row contains header names like 'Area' and 'Asset', use it as header
    first_row = df.iloc[0].astype(str).str.strip()
    if {'Area', 'Asset'}.issubset(set(first_row.values)):
        df.columns = first_row
        df = df.drop(df.index[0]).reset_index(drop=True)

    # Replace empty strings with NA and drop rows that are entirely NA
    df = df.replace(r'^\s*$', pd.NA, regex=True).dropna(axis=0, how='all').reset_index(drop=True)

    # Strip whitespace from object columns
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).str.strip()

    # Convert known numeric columns to integers where possible
    numeric_cols = ['Land price', 'House price', 'Rent: 0', 'Rent: 1', 'Rent: 2', 'Rent: 3', 'Rent: 4']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Select only the required columns for asset structure
    required_columns = ['Area', 'Asset', 'Land price', 'House price', 'Rent: 0', 'Rent: 1', 'Rent: 2', 'Rent: 3', 'Rent: 4']
    existing_cols = [col for col in required_columns if col in df.columns]
    df = df[existing_cols].copy()

    # Drop rows with any NaN values in the selected columns and reset index
    df = df.dropna(how='any').reset_index(drop=True)

    # Build the assets JSON from the cleaned dataframe with Area as primary key
    assets = {}
    for _, row in df.iterrows():
        area = str(row['Area']).strip()
        asset_name = str(row['Asset']).strip()
        if area not in assets:
            assets[area] = {}
        assets[area][asset_name] = {
            "land_price": int(row['Land price']),
            "house_price": int(row['House price']),
            "rent": {
                "no_houses": int(row['Rent: 0']),
                "one_house": int(row['Rent: 1']),
                "two_houses": int(row['Rent: 2']),
                "three_houses": int(row['Rent: 3']),
                "four_houses": int(row['Rent: 4'])
            }
        }

    # Determine JSON filename if not provided and write file
    if json_file is None:
        json_file = os.path.splitext(excel_file)[0] + '.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(assets, f, indent=2)


In [20]:
excel_to_assets_json("Asset_database.xlsx")

In [3]:
# Test Case: Attempting to assign an asset that's already owned by another player
print("=== Test Case: Asset Conflict Detection with Area-Asset Hierarchy ===\n")

# Define test database file paths
test_db_file = "Player_database.json"
asset_db_file = "Asset_database.json"

# Remove test file if it exists from previous test run
if os.path.exists(test_db_file):
    os.remove(test_db_file)
    print(f"Removed old test database file: {test_db_file}\n")

# First, generate the asset database from Excel if it doesn't exist
if not os.path.exists(asset_db_file):
    print(f"Generating asset database from Excel...\n")
    excel_to_assets_json("Asset_database.xlsx", asset_db_file)

# Load and display available areas and assets
with open(asset_db_file, 'r', encoding='utf-8') as f:
    asset_db = json.load(f)

print("Available Areas and Assets in Asset Database:")
print("-" * 50)
for area, assets in asset_db.items():
    print(f"Area: {area}")
    for asset_name in assets.keys():
        print(f"  - {asset_name}")
print()

print("Step 1: Assigning assets to Player 1")
print("-" * 50)
assign_asset_to_player(test_db_file, player_name="Player 1", area_name=list(asset_db.keys())[0], asset_name=list(asset_db[list(asset_db.keys())[0]].keys())[0], houses=4, asset_database_file=asset_db_file)
assign_asset_to_player(test_db_file, player_name="Player 1", area_name=list(asset_db.keys())[0], asset_name=list(asset_db[list(asset_db.keys())[0]].keys())[1], houses=3, asset_database_file=asset_db_file)

print("\nStep 2: Assigning assets to Player 2")
print("-" * 50)
if len(asset_db) > 1:
    assign_asset_to_player(test_db_file, player_name="Player 2", area_name=list(asset_db.keys())[1], asset_name=list(asset_db[list(asset_db.keys())[1]].keys())[0], houses=1, asset_database_file=asset_db_file)
    assign_asset_to_player(test_db_file, player_name="Player 2", area_name=list(asset_db.keys())[1], asset_name=list(asset_db[list(asset_db.keys())[1]].keys())[1], houses=0, asset_database_file=asset_db_file)
else:
    assign_asset_to_player(test_db_file, player_name="Player 2", area_name=list(asset_db.keys())[0], asset_name=list(asset_db[list(asset_db.keys())[0]].keys())[2], houses=1, asset_database_file=asset_db_file)
    assign_asset_to_player(test_db_file, player_name="Player 2", area_name=list(asset_db.keys())[0], asset_name=list(asset_db[list(asset_db.keys())[0]].keys())[3], houses=0, asset_database_file=asset_db_file)

# Load and display current database state
with open(test_db_file, 'r', encoding='utf-8') as f:
    test_db = json.load(f)

print("\nCurrent database state:")
print(json.dumps(test_db, indent=2))

print("\n" + "="*50)
print("Step 3: Testing Conflict - Try to assign asset from Player 1 to Player 2")
print("-" * 50)
area1 = list(asset_db.keys())[0]
asset1 = list(asset_db[area1].keys())[0]
assign_asset_to_player(test_db_file, player_name="Player 2", area_name=area1, asset_name=asset1, houses=2, asset_database_file=asset_db_file)

print("\n" + "="*50)
print("Step 4: Testing House Cap - Try to assign more than 4 houses")
print("-" * 50)
assign_asset_to_player(test_db_file, player_name="Player 1", area_name=area1, asset_name=asset1, houses=5, asset_database_file=asset_db_file)

print("\n" + "="*50)
print("Step 5: Testing House Addition - Add houses to existing asset (Player 1)")
print("-" * 50)
assign_asset_to_player(test_db_file, player_name="Player 1", area_name=area1, asset_name=asset1, houses=2, asset_database_file=asset_db_file)

print("\n" + "="*50)
print("Step 6: Testing Invalid Asset - Try to assign non-existent asset")
print("-" * 50)
assign_asset_to_player(test_db_file, player_name="Player 1", area_name=area1, asset_name="Non-existent Asset", houses=2, asset_database_file=asset_db_file)

# Load and display final database state
with open(test_db_file, 'r', encoding='utf-8') as f:
    test_db = json.load(f)

print("\n" + "="*50)
print("Final database state (with accumulated houses):")
print(json.dumps(test_db, indent=2))

# Verify no duplicates exist
print("\n" + "="*50)
print("Verification: Checking for duplicate assignments")
print("-" * 50)
all_assets = set()
has_duplicates = False
for player, areas in test_db.items():
    for area, assets in areas.items():
        for asset in assets:
            asset_key = f"{area}::{asset}"
            if asset_key in all_assets:
                print(f"DUPLICATE FOUND: {asset} in area {area} is assigned to multiple players!")
                has_duplicates = True
            else:
                all_assets.add(asset_key)

if not has_duplicates:
    print("✓ No duplicate assignments detected. All assets are uniquely assigned.")
else:
    print("✗ Duplicate assignments detected!")

=== Test Case: Asset Conflict Detection with Area-Asset Hierarchy ===

Removed old test database file: Player_database.json

Available Areas and Assets in Asset Database:
--------------------------------------------------
Area: Street 1
  - Asset 1.1
  - Asset 1.2
  - Asset 1.3
  - Asset 1.4
Area: Street 2
  - Asset 2.1
  - Asset 2.2
  - Asset 2.3
  - Asset 2.4

Step 1: Assigning assets to Player 1
--------------------------------------------------
INFO: Created new player database (file 'Player_database.json' did not exist)
SUCCESS: Asset 'Asset 1.1' (Street 1) assigned to player 'Player 1' with 4 houses
INFO: Player database saved to 'Player_database.json'
INFO: Loaded player database from 'Player_database.json'
SUCCESS: Asset 'Asset 1.2' (Street 1) assigned to player 'Player 1' with 3 houses
INFO: Player database saved to 'Player_database.json'

Step 2: Assigning assets to Player 2
--------------------------------------------------
INFO: Loaded player database from 'Player_database.